### 数据预处理

#### 1. 根据语料分别构建源数据词表 和 目标数据词表

In [ ]:
import sys 
sys.path.append("..")
from Utils.Dictionary import Dictionary


data_path = "../Data/Dataset/data.csv"
source_dic_path = "../Data/source_vocal.pkl"
target_dic_path = "../Data/target_vocal.pkl"


# 生成 源数据字典 and 目标数据字典
source_dic = Dictionary()
source_dic.load_from_data(data_path, "raw_data")
source_dic.dump_to_file(source_dic_path)

target_dic = Dictionary()
target_dic.load_from_data(data_path, "raw_data")
target_dic.dump_to_file(target_dic_path)
target_dic.word2idx

#### 2. 数据集的划分（如果已经划分好就不必划分）

In [ ]:
import pandas as pd

# 文件路径
data_path = "../Data/Dataset/data.csv"
train_data_path = "../Data/Dataset/train_data.csv"
test_data_path = "../Data/Dataset/test_data.csv"

# 数据集划分
split_ratio = 0.8
dataset = pd.read_csv(data_path, index_col=0)
train_set = dataset.sample(frac = split_ratio, random_state=0, axis=0)
test_set = dataset[~dataset.index.isin(train_set.index)]

# 数据集保存
train_set.to_csv(train_data_path)
test_set.to_csv(test_data_path)


#### 数据增强

In [9]:
import random
from Utils.VariantNeedleman_Wunsch import VariantNW
import pandas as pd
import pickle
import json

# 寻找可疑变异
def findVariant(seq_1, seq_2):
    variant_dic = {}
    for variant, right in zip(seq_1, seq_2):
        
        if isVariant(variant, right):
            if right not in variant_dic.keys():
                value = [variant]
                variant_dic[right] = value
            else:
                value = variant_dic[right]
                value.append(variant)
                variant_dic[right] = value

    return variant_dic

# 筛选变体字典
def filterVariant(variant_dic, remove_AandD = True, remove_single = True, remove_ratio = 0.5):
    variant_dic_2 = variant_dic.copy()

    for key, value in variant_dic.items():

        if (key.encode("utf-8").isalpha() or key.isdigit()) and remove_AandD:
            delete_key = True

        if len(variant_dic[key]) == 1 and remove_single:
            delete_key = True
        
        if delete_key:
            del variant_dic_2[key]
        else:
            for val in value:
                variantNW.set_seqs(key, val)
                variantNW.propagate()
                aligned_seq1, aligned_seq2 = variantNW.traceback()
                score = variantNW.get_aligned_seq_score(aligned_seq1, aligned_seq2).pop()
                if score < remove_ratio:
                    variant_dic[key].remove(val)
                
    return variant_dic_2


# 判断是否是变异
def isVariant(char1, char2):
    if char1 == char2 or char1 == '[GAP]' or char2 == '[GAP]':
        return False
    else:
        return True

# 生成变异数据
def genVariant(variant_dic, seq_1, seq_2):
    
    raw_seq_1 = seq_1.copy()

    for variant, right in zip(seq_1, seq_2):
        
        if isVariant(variant, right):
            if right in variant_dic.keys():
                a = random.choice(variant_dic[right])
                while a == variant:
                    a = random.choice(variant_dic[right])
                seq_1[seq_1.index(variant)] = a

    if seq_1 == raw_seq_1:
        return 0, 0
    else:
        return seq_1, seq_2


variantNW = VariantNW()

# 读取数据
data = pd.read_csv("../Data/Dataset/train_data.csv", index_col=0).reset_index(drop=True)
with open("../Data/variant_dic_remove_re.json", "r", encoding='UTF-8') as f:
    variant_dic = json.load(f)


raw_data = []
right_data = []
for row in data.iterrows():
    seq1 = row[1][0]
    seq2 = row[1][1]
    variantNW.set_seqs(seq1, seq2)
    variantNW.propagate()
    aligned_seq1, aligned_seq2 = variantNW.traceback()
    seq1,seq2 = genVariant(variant_dic, aligned_seq1, aligned_seq2)
    if seq1 == 0:
        continue
    else:
        try:
            seq1 = "".join(seq1).replace("[GAP]","")
            seq2 = "".join(seq2).replace("[GAP]","")
        except:
            pass
        if row[0] % 1000 == 0:
            print(1)
        raw_data.append(seq1)
        right_data.append(seq2)
dataframe = pd.DataFrame({"raw_data":raw_data, "right_data":right_data})
dataframe

1
1
1
1
1
1


,raw_data,right_data
0,权益服务：您领取的双突战法指标今天已有信号提醒啦譽快登录大智慧b查看或现在打开链接h://t...,"权益服务:您领取的双突战法指标今天已有信号提醒啦,快登录大智慧b查看或现在打开链接h://t..."
1,【亲朋萁牌 】来就送68-668！忸ν 忸主主 砟金?更多好玩等你来 e5.q00k....,【亲朋棋牌】来就送68-668!牛牛斗地主炸金更多好玩等你来e5.q00k.com29634...
2,"告诉黄某某,速速回家,明天有外访工作人员去家里核实 豆 豆 qian芡 单情况,微信：ht...","告诉黄某某,速速回家,明天有外访工作人员去家里核实豆豆欠账单情况,微信:htsz6666"
3,"你好!天意如此,让你遇见,玷： 18535.c0 领18-188!","你好!天意如此,让你遇见,点:18535.c0领18188!"
4,授信成功：您7帐号获5首批提现资格镖余额6元譽提现s.s/q?8r2谨防泄露退订回x,"授信成功:您7帐号获5首批提现资格,余额6元,提现s.s/q?8r2谨防泄露退订回x"
...,...,...
7811,玺诺琥珀：除夕狂欢夜镖今晚漏到爆！珠宝壕礼送到你手软！点7q.o/k5t领取牛气大礼包！（阈...,"玺诺琥珀:除夕狂欢夜,今晚漏到爆!珠宝壕礼送到你手软!点7q.o/k5t领取牛气大礼包!(阈..."
7812,哇⁢(_3期必中洄血纪划⁢注册就送一百啟冻金 加Ｖ縅解：fp18860274159惦cDw,"哇,(3期必中回血计划,注册就送一百启动金加微了解:fp18860274159.cDw"
7813,公司贷款有返点孒商贷返千1-1.5抵押贷款150以上千8垫资2个点垫首付4个点2变首4泇3贷...,公司贷款有返点了商贷返千1-1.5抵押贷款150以上千8垫资2个点垫首付4个点2变首4加3贷...
7814,"阁下,老哥,你号一水不限攸戯,珫100到137,丛1000到1188,加Q 24773942...","阁下,老哥,你号一水不限游戏,充100到137,充1000到1188,加Q247739423..."


In [10]:
dataframe.to_csv("../Data/Dataset/train_data_supply_0409.csv")

In [15]:
import pandas as pd
data_1 = pd.read_csv("../Data/Dataset/train_data_supply_0409.csv", index_col=0).sample(frac=0.15, axis=0)
data_2 = pd.read_csv("../Data/Dataset/train_data.csv", index_col=0)
data_1 = pd.concat([data_1, data_2], axis=0, ignore_index=True)
data_1.to_csv("../Data/Dataset/train_data_supply.csv")